# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [2]:
df = pd.read_csv('/datasets/data.csv') # читаем файл /datasets/data.csv и сохраняем его как df
display(df.head(5))
print() # разделим пустой строчкой
df.info() # выводим на экран информацию о файле

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Посмотрим наименование столбцов, все ли в них правильно. 

In [3]:
print(df.columns)

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


Название столбцов правильное. Все корректно - ошибок нет

Посмотрим сводные данные по файлу.

In [4]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Количество строк - 21525, 12 столбцов.
- По столбцу children (количество детей в семье) видим отрицательное значение и количество детей 20.
- По стообцу days_employed (общий трудовой стаж в днях) видим отрицательные значения, может быть значения не одного формата (есть 401755 (если в днях более 1100 дней), -291, -18388), и пустые значения (19351 строчка, вместо 21525)
- По столбцу dob_years (возраст клиента в годах) все хорошо.
- По столбцу education (уровень образования клиента)- видим использование разных регистров.
- По столбцу education_id (идентификатор уровня образования) - все хорошо.
- По столбцу family_status_id (family_status_id) - все хорошо.
- По столбцу family_status (семейное положение) - не видно замечаний. 
- По столбцу gender (пол клиента) - не видим замечаний.
- По столбцу income_type (income_type) - не видим замечаний.
- По столбцу debt (имел ли задолженность по возврату кредитов) - не видим замечаний.
- По столбцу total_income (ежемесячный доход) - видим формат с плавающей точкой, насколько он необходим.
- По столбцу purpose (цель получения кредита) - типы кредитов не записаны корректно, есть одинаковые цели, но называются по разному

Данные требуют корректировки.

### Обработка пропусков

In [5]:
df.isnull().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Посмотрели суммарное количество пропусков - видим в столбце days_employed (трудовой стаж) и total_income (ежемесячный доход) 2174 пустых значения. Это примерно 10% и надо выяснять причину отсутствия данных.

Пропущенные значения в столбце доход, будем смотреть по медианному значению от типа занятости. 
Посмотрим, где пропущены значения по зарплате

In [6]:
df.loc[df['total_income'].isnull(), 'income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Найдем среднее по этим категориям

In [7]:
median_total_income = df.groupby('income_type')['total_income'].median()
round(median_total_income) # округляем значения по математическим правилам

income_type
безработный        131340.0
в декрете           53829.0
госслужащий        150448.0
компаньон          172358.0
пенсионер          118514.0
предприниматель    499163.0
сотрудник          142594.0
студент             98202.0
Name: total_income, dtype: float64

Заменим значения в столбце 

In [8]:
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'сотрудник'), 'total_income'] = median_total_income[6]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'компаньон'), 'total_income'] = median_total_income[3]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'пенсионер'), 'total_income'] = median_total_income[4]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'госслужащий'), 'total_income'] = median_total_income[2]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'предприниматель'), 'total_income'] = median_total_income[5]
df.isnull().sum() # проверяем выполнение замены

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Посмотрим какие есть значения в столбце возраст

In [9]:
df['dob_years'].value_counts().sort_index(ascending=True)

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Видим не правильное значение - 0 лет

Будем считать возраст от медианного возраста по категории тип занятости, а также посмотрим категории, где есть пустые значения.

In [10]:
df.loc[df['dob_years'] == 0, 'income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

Вычислим медианный возраст в категори тип занятости

In [11]:
age_median = df.groupby('income_type')['dob_years'].median()
age_median

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

Заменим нулевые значения на медианы и проверим все ли нули исправили. 

In [12]:
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'сотрудник'), 'dob_years'] = age_median[6]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'пенсионер'), 'dob_years'] = age_median[4]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'компаньон'), 'dob_years'] = age_median[3]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'госслужащий'), 'dob_years'] = age_median[2]

df['dob_years'].value_counts().sort_index(ascending=True)

19.0     14
20.0     51
21.0    111
22.0    183
23.0    254
24.0    264
25.0    357
26.0    408
27.0    493
28.0    503
29.0    545
30.0    540
31.0    560
32.0    510
33.0    581
34.0    603
35.0    617
36.0    555
37.0    537
38.0    598
39.0    648
40.0    615
41.0    607
42.0    597
43.0    513
44.0    547
45.0    497
46.0    475
47.0    480
48.0    538
49.0    508
50.0    514
51.0    448
52.0    484
53.0    459
54.0    479
55.0    443
56.0    487
57.0    460
58.0    461
59.0    444
60.0    397
61.0    355
62.0    352
63.0    269
64.0    265
65.0    194
66.0    183
67.0    167
68.0     99
69.0     85
70.0     65
71.0     58
72.0     33
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

Посмотрим количество значений по столбцу дети. 

In [13]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видим, что у 76 клиентов 20 детей, считаем, что это ошибка и меняем это число на 2
Значение -1 вероятно является ошибкой, заменим на 1.
И опять предложим коллегам внести изменение в программу для внесения записей о клиентах, выбирать из словаря и проверка на дурака - то есть когда более 3 детей, чтобы программа уточняла - точно столько-то детей?

In [14]:
df.loc[df['children'] == 20, 'children'] = 2    # указыаем, что 20 в столбце children = 2
df.loc[df['children'] == -1, 'children'] = 1    # указыаем, что -1 в столбце children = 1
df['children'].value_counts() # проверяем результат

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Посмотрим значения в столбеце dob_years и сортируем по возростанию.
Значения в столбце 'days_employed' требует уточнения у сотрудников, кто его дал. Значения достигают 1018 лет, если значения предоставлены в днях.

In [15]:
df.loc[df['dob_years'] == 0, 'days_employed'].value_counts().sort_index(ascending=True) 

Series([], Name: days_employed, dtype: int64)

Найдем стаж "days_employed" по медианному значению возроста.
Значения в годах у нас от 19 до 75 лет (75-19=56, разделим на 4 группы по 14 лет)
1 группа 19 - 33
2 группа 34 - 47
3 группа 48 - 61
4 группа 62 - 75

In [16]:
def days_employed(row):                           # поставим условия для обозначения групп
    
    age = row['dob_years']

    if age <= 33:
        return '1 группа'
    
    if age <= 47:
        return '2 группа'
    
    if age <= 61:
        return '3 группа'
    
    if age > 61:
        return '4 группа'

df['age_group'] = df.apply(days_employed, axis=1)
                              
display(df.head())  #  применяем и смотрим, что получилось

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,2 группа
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,2 группа
2,0,-5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1 группа
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,1 группа
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,3 группа


Найдем медианное значение для каждой группы

In [17]:
median_days_employed = df.groupby('age_group')['days_employed'].median()
print(median_days_employed)

age_group
1 группа     -1130.581631
2 группа     -1802.397660
3 группа      -922.040893
4 группа    358174.951271
Name: days_employed, dtype: float64


Заменяем значения

In [18]:
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == '1 группа'), 'days_employed'] = median_days_employed[0]
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == '2 группа'), 'days_employed'] = median_days_employed[1]
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == '3 группа'), 'days_employed'] = median_days_employed[2]
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == '4 группа'), 'days_employed'] = median_days_employed[3]
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,59634.322926,43.496167,0.817236,0.972544,0.080883,1.652253e+05
std,0.755528,137656.024073,12.231538,0.548138,1.420324,0.272661,9.804367e+04
min,0.000000,-18388.949901,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1143.420356,43.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,-351.114070,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

- Произвели поиск пропусков и обнаружили одиниковое количесвто пропусков, по 2174 в столбцах "total_income" и "days_employed".
- Произвели обработку пропусков в "total_income" используя медиану по типу занятости.
- В столбце возраст обнаружили 101 пустое значение, которые заполнили по медианному значению  от типа занятости.
- В столбце дети заменили значения -1 и 20 на 1 и 2 применив замену.
- Пропущенные значения "days_employed" заполнили по медианyому значению возраста.
- В этих случаях заполняли по медианному значениях, так как применение "среднего" не было бы корректным из-за слишком большого разброса данных.
- Пропуски получены или из-за незаполненности базы данных или это ошибка при выгрузке

## Предобработка данных

### Замена типа данных

В столбце days_employed делаем значения числовыми и положительными 

In [19]:
df['days_employed'] = df['days_employed'].astype('int')  # меняем тип на целый числовой
df['days_employed'] = abs(df['days_employed'])           # делаем их положительными

display(df.head())                                       # проверяем, что получилось

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,2 группа
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,2 группа
2,0,5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1 группа
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,1 группа
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,3 группа


В значениях столбца total_income нет необходимости в таких точных числах, округлим их.

In [20]:
df["total_income"] = df["total_income"].astype(int)

In [21]:
df.info() # получаем сведения о таблице.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ MB


Видим, что два столбца с числовыми значениями имеют тип 'object' - days_employed и total_income

**Вывод**

Анализ таблицы показал, что изначально у 2 столбцов не правильный формат данных (строковый, а не числовой). Заменили тип на числовые целые, так как значения с десятичными значениями не требуется для решения задач. 

### Обработка дубликатов

Посмотрим количесвто дубликатов

In [22]:
df.duplicated().sum()

54

Удалим дубликаты используя метод drop_duplicates() с новой индексацией

In [23]:
df = df.drop_duplicates().reset_index(drop=True)

Начнем проверять столбы со строковым значением на наличие дубликатов с учетом регистра

In [24]:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Сделаем всё строчными буквы

In [25]:
df['education'] = df['education'].str.lower() 
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [26]:
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [27]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [28]:
df['income_type'].value_counts() # проверили, все нормально. Категорий, которые можно было бы объединит с другими нет.

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Посмотрели, все корректно. Категорий, которые можно было бы объединит с другими нет.

**Вывод**

- Убрали дубликаты строк - 54 штуки. Возможно это вызвано ошибками при выгрузки файла или было задвоение человека в системе.
- Проверили столбцы со строковыми значениями на наличие дубликатов. Так как было применины разные регистры, то применили метод 
str.lower() для приведения к нормальному виду. 

Появление ошибок произошло из-за человеского фактора и настройки программы - нужно подсказать коллегам из того отдела, чтобы нужно было выбирать значение, а не вводить его руками.
Надо проверить всю базу по клиентам на наличие в ней дуликатов, но уже с большей информацией - паспортные данные.

### Лемматизация

Для лемматизации (будем работать со столбцом цели кредита) будем использовать библиотеку pymystem3 и контейнер Counter

Посчитаем все варианты целей кредита.

In [29]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Указываем слова, по которым будем искать

In [30]:
words = ["сдача", "коммерческий", "жилье", "образование", "свадьба", "недвижимость", "автомобиль"]

Применяем леммитизацию и заменяем значения в строках на основное слово 

In [31]:
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in words:
        if word in lemma:
            lemma = word
    return lemma

df['purpose_group'] = df['purpose'].apply(lemmatize)        
df['purpose_group'].value_counts()

недвижимость    5041
автомобиль      4308
образование     4014
жилье           3809
свадьба         2335
коммерческий    1312
сдача            652
Name: purpose_group, dtype: int64

Объединяем похожие значения строк: жилье/недвижимость, коммерческий/недвижимость, сдача/недвижимость

In [32]:
df.loc[df['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'
df.loc[df['purpose_group'] == 'коммерческий', 'purpose_group'] = 'недвижимость'
df.loc[df['purpose_group'] == 'сдача', 'purpose_group'] = 'недвижимость'
df['purpose_group'].value_counts()

недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose_group, dtype: int64

In [33]:
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,2 группа,недвижимость
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2 группа,автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1 группа,недвижимость
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1 группа,образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,3 группа,свадьба


**Вывод**

Провели леммитизацию используя библиотеку pymystem3 и контейнер Counter. 
Получили 4 цели получения кредита: 
- недвижимость, 
- автомобиль, 
- образование,
- свадьба.

### Категоризация данных

Необходимо категоризировать столбец уровень дохода, для решения поставленной задачи. Посмотрим первые и посление значения

In [34]:
df['total_income'].value_counts().sort_index(ascending=True)

20667      1
21205      1
21367      1
21695      1
21895      1
          ..
1711309    1
1715018    1
1726276    1
2200852    1
2265604    1
Name: total_income, Length: 18608, dtype: int64

Разброс очень большой. Посмотрим медиану

In [35]:
age_median = df['total_income'].median()
print(age_median)

142594.0


Для правильно разбиения на группы применим метод qcut, который поможет определить количество квантилей и позволить pandas разделить данные. В качестве аргумента укажем 4, как количество групп, которые необходимо поделить. Эти сведения также можно взять из df.describe() - там это 25%, 50%, 75%

In [36]:
pd.qcut(df['total_income'], q=4)

0        (195767.5, 2265604.0]
1         (107654.5, 142594.0]
2         (142594.0, 195767.5]
3        (195767.5, 2265604.0]
4         (142594.0, 195767.5]
                 ...          
21466    (195767.5, 2265604.0]
21467     (142594.0, 195767.5]
21468    (20666.999, 107654.5]
21469    (195767.5, 2265604.0]
21470    (20666.999, 107654.5]
Name: total_income, Length: 21471, dtype: category
Categories (4, interval[float64]): [(20666.999, 107654.5] < (107654.5, 142594.0] < (142594.0, 195767.5] < (195767.5, 2265604.0]]

Сделаем 4 группы доходов: 
- 0 - 108
- 108 - 142
- 142 - 195
- 195 >

In [37]:
def total_income(row):                           # поставим условия для обозначения групп
    
    median_total_income = row['total_income']

    if median_total_income <= 108000:
        return '0 - 108'
    
    if median_total_income <= 142000:
        return '108 - 142'
    
    if median_total_income <= 195000:
        return '142 - 195'
    
    if median_total_income > 195000:
        return 'больше 195'

df['median_total_income'] = df.apply(total_income, axis=1)
                              
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group,median_total_income
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,2 группа,недвижимость,больше 195
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2 группа,автомобиль,108 - 142
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1 группа,недвижимость,142 - 195
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1 группа,образование,больше 195
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,3 группа,свадьба,142 - 195


In [38]:
df['age_group'].value_counts()

2 группа    7870
3 группа    6455
1 группа    5368
4 группа    1778
Name: age_group, dtype: int64

### Выделение словаря

В наших данных есть словари (Словарь — неупорядоченная структура данных, которая позволяет хранить пары «ключ — значение»). Выделим их отдельно.

Видим с нашем файле два словаря, это ключи family_status_id, education_id и значение: столбец family_status и education.
Вынесем данные по этим параметрам попарно в отдельный датафрейм, а затем удалим встречающиеся дубликаты. Выделение словарей с удалением текстовых параметров из основной таблицы поможет нам увеличить эффективность работы с данными из-за уменьшения веса самого датасета. 

In [39]:
df_slovar_1 = df[['education','education_id']]
df_slovar_2 = df[["family_status", 'family_status_id']]
display(df_slovar_1.head(6))
print(df_slovar_1.duplicated().sum())
df_slovar_1 = df_slovar_1.drop_duplicates().reset_index(drop=True)


,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
5,высшее,0


21466


In [40]:
display(df_slovar_2.head(6))
print(df_slovar_2.duplicated().sum())
df_slovar_2 = df_slovar_2.drop_duplicates().reset_index(drop=True)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
5,гражданский брак,1


21466


In [41]:
df_slovar_1

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [42]:
df_slovar_2

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


**Вывод** 

Для решения поставленной задачи необходимо категоризивовать данные по столбцу общий доход.
Для этого нашли медианну и поделили еще на 2 группы внутри.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
df['debt'].value_counts() # смотрим общее количесвто просрочек по кредиту - 1741

0    19730
1     1741
Name: debt, dtype: int64

In [44]:
df_pivot_children = df.pivot_table(index='children', values='debt', aggfunc=['count','sum'])                     # строим сводную таблицу 
df_pivot_children['%']=round((df_pivot_children[('sum', 'debt')]/df_pivot_children[('count', 'debt')])*100,2)    # округляем и считаем проценты по соотношению столбцов 
df_pivot_children.columns = ['Количество детей', "Количество просрочек по кредиту", "% просрочек"]               # переименовываем столбцы
print(df_pivot_children.sort_values(by = "% просрочек", ascending = False).head(10))                             # сортируем по убыванию столбец с процентным соотношением

          Количество детей  Количество просрочек по кредиту  % просрочек
children                                                                
4                       41                                4         9.76
2                     2128                              202         9.49
1                     4856                              445         9.16
3                      330                               27         8.18
0                    14107                             1063         7.54
5                        9                                0         0.00


**Вывод**

Количество детей приводит к увеличению просрочек по кредиту + 2,2 % (9.76 - 7.54). Видимо это связно с тратами, связанными с содержанием детей

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [45]:
df_pivot_family_status = df.pivot_table(index=['family_status'], values='debt', aggfunc=['count','sum']) #
df_pivot_family_status['%']=round((df_pivot_family_status[('sum', 'debt')]/df_pivot_family_status[('count', 'debt')])*100,2)
df_pivot_family_status.columns = ['Количество клиентов', "Количество просрочек", "% просрочек"]
print(df_pivot_family_status.sort_values(by = "% просрочек", ascending = False).head(10))                    

                       Количество клиентов  Количество просрочек  % просрочек
family_status                                                                
не женат / не замужем                 2810                   274         9.75
гражданский брак                      4163                   388         9.32
женат / замужем                      12344                   931         7.54
в разводе                             1195                    85         7.11
вдовец / вдова                         959                    63         6.57


**Вывод**

- Количество просрочек по кредиту меньше всего у кетегории 'вдовец / вдова', и тех, кто в "разводе" - это вызвано, тем, что траты сократились и пережитой стрессовой ситуацией. 
- Больше всего у тех, кто "не женат / не замужем" или находится в 'гражданский брак' - причина в тратах на поиски половинки и более беззаботной жизни, когда не отвечают ни за кого.  
- Просрочка у 'женат / замужем' находится на среднем уровне. 



- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [46]:
df_pivot_income = df.pivot_table(index='median_total_income', values='debt', aggfunc=['count','sum']) #
df_pivot_income['%']=round((df_pivot_income[('sum', 'debt')]/df_pivot_income[('count', 'debt')])*100,2)
df_pivot_income.columns = ['Количество кредитов', "Количесвто просрочек", "% просрочек"]
print(df_pivot_income.sort_values(by = "% просрочек", ascending = False).head(10))

                     Количество кредитов  Количесвто просрочек  % просрочек
median_total_income                                                        
142 - 195                           6340                   550         8.68
108 - 142                           4296                   371         8.64
0 - 108                             5411                   431         7.97
больше 195                          5424                   389         7.17


**Вывод**

Уровень дохода влияет на количесвто просрочек по кредиту.
- Больше всего просрочек (8.68%) у клиентов с достатком от 142 - 195 тыс.руб. - у среднего класса.
- Меньше всего просрочек (7.17%) у клиентов с доходом более 195 тыс.руб.

- Как разные цели кредита влияют на его возврат в срок?

In [47]:
df_pivot_purpose = df.pivot_table(index='purpose_group', values='debt', aggfunc=['count','sum']) #
df_pivot_purpose['%']=round((df_pivot_purpose[('sum', 'debt')]/df_pivot_purpose[('count', 'debt')])*100,2)
df_pivot_purpose.columns = ['Количество кредитов', "Количесвто просрочек", "% просрочек"]
print(df_pivot_purpose.sort_values(by = "% просрочек", ascending = False).head(10))

               Количество кредитов  Количесвто просрочек  % просрочек
purpose_group                                                        
автомобиль                    4308                   403         9.35
образование                   4014                   370         9.22
свадьба                       2335                   186         7.97
недвижимость                 10814                   782         7.23


**Вывод**

Больше всего просрочек допускается при покупке автомобиля, возможно от того, что это не является средством первой необходимости.
Образование находится на втором месте по просрочкам, возможно из-за того, что не сразу удается найти работу после обучения или во время учебы.
Просрочека на свадьбу 7.97% (3-е место), возможно, что кредит еще выплачивается во время подготовки к свадьб. 
Меньше всего просрочек по недвижимости, так как есть риск потерять его

## Общий вывод

Необходимо предложить коллегам из отдела настройки ПО, что для избежания ошибок сделать выбор целей кредита из "словаря"; при введении более 3-х детей, уточняющее всплывающее окно и значения могут быть только положительные; при указании стажа, также только положительные числа и проверка на срок (если более 56 лет стажа (75 (самый большой возраст в нашем файле) - 19 (самый молодой в нашем файле)) также уточняющее окно); поле образование - также заполнять из "словаря".

Полученный файл поддкорректировали и выявили зависимость от количества детей, семейного состояния, уровня дохода и целей кредита на задолженность по возврату кредита.

Идеальный заемщик для банка это бездетный (или с 5-ю детьми) семейный (состоящий в официальном браке, разведенный или вдовец/вдова), с доходом более 195 тыс.руб., клиент, который берет кредит на недвижимость или свадьбу.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [X]  открыт файл;
- [X]  файл изучен;
- [X]  определены пропущенные значения;
- [X]  заполнены пропущенные значения;
- [X]  есть пояснение, какие пропущенные значения обнаружены;
- [X]  описаны возможные причины появления пропусков в данных;
- [X]  объяснено, по какому принципу заполнены пропуски;
- [X]  заменен вещественный тип данных на целочисленный;
- [X]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [X]  удалены дубликаты;
- [X]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [X]  описаны возможные причины появления дубликатов в данных;
- [X]  выделены леммы в значениях столбца с целями получения кредита;
- [X]  описан процесс лемматизации;
- [X]  данные категоризированы;
- [X]  есть объяснение принципа категоризации данных;
- [X]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [X]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [X]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [X]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [X]  в каждом этапе есть выводы;
- [X]  есть общий вывод.